In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.io import read_image
import cv2
from matplotlib import pyplot as plt
from zipfile import ZipFile
import os
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df=pd.read_csv('train_cv_comp.csv')
train_y=torch.tensor(train_df['label'])


In [ ]:
X_test=torch.empty((2136,3,256,256))
images_folder = '/content/drive/MyDrive/test_256'

transform = transforms.Compose([
    transforms.ToPILImage(),  # Конвертируем numpy array в PIL Image
    transforms.Resize((224, 224)),  # Изменяем размер (опционально)
    transforms.ToTensor(),  # Конвертируем в тензор [0,1]
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация для ImageNet
])


for i,filename in enumerate(os.listdir(images_folder)):
    img_path = os.path.join(images_folder, filename)

    # Чтение изображения с помощью OpenCV
    img = cv2.imread(img_path)
    if img is not None:
        # Конвертация BGR (OpenCV) в RGB (для matplotlib)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # tensor_img = transform(img_rgb)
        X_test[i]=torch.tensor(img_rgb.transpose(2,0,1))
    else:
        print(filename)

In [ ]:
X_train=torch.empty((6424,3,256,256))
images_folder_train = '/content/drive/MyDrive/train_256'


for i,filename in enumerate(os.listdir(images_folder_train)):
    img_path = os.path.join(images_folder_train, filename)

    # Чтение изображения с помощью OpenCV
    img = cv2.imread(img_path)

    if img is not None:
        # Конвертация BGR (OpenCV) в RGB (для matplotlib)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # tensor_img = transform(img_rgb)
        X_train[i]=torch.tensor(img_rgb.transpose(2,0,1))
    else:
        print(filename)

In [ ]:
import timm

# Загрузка предобученной Swin-Large для 224x224
model = timm.create_model(
    "swin_large_patch4_window7_224",  # название модели для 224x224
    pretrained=True,                  # загрузить веса, обученные на ImageNet-1K
    num_classes=46,                 # число классов (можно изменить под свою задачу)
)

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader,TensorDataset

import torchvision.transforms.functional as F
from torch import nn

transform = transforms.RandomRotation(degrees=(0, 360))

def train_model(
    model,
    X_train,
    Y_train,
    X_test,
    Y_test,
    criterion,
    optimizer,
    num_epochs=10,
    device='cuda',
    batch_size=16,
    with_cv=False
  ):

    train_data=TensorDataset(X_train,Y_train)
    train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=True)
    test_data=TensorDataset(X_test,Y_test)
    val_loader=DataLoader(test_data,batch_size=batch_size,shuffle=False)
    model = model.to(device)
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Прогресс-бар для обучения
        train_loop = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for inputs, labels in train_loop:
            inputs=transform(inputs)
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Обновление прогресс-бара
            train_loop.set_postfix(loss=running_loss/(total/len(train_loader.dataset)),
                                 acc=100.*correct/total)

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total

        # Валидация
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        # Логирование
        print(f'Epoch {epoch+1}: '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% |'
              f'Train Loss: {train_loss:.4f}, Train Acc: {val_acc:.2f}%'
        )

    print('Training complete!')
    return model

def validate(model, val_loader, criterion, device):
    """Функция валидации"""
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        val_loop = tqdm(val_loader, desc='Validating')
        for inputs, labels in val_loop:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            val_loop.set_postfix(loss=val_loss/(total/len(val_loader.dataset)),
                               acc=100.*correct/total)

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct / total
    return val_loss, val_acc


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, train_y, test_size=0.2, random_state=42)

In [ ]:
for param in model.parameters():
    param.requires_grad = False
# for param in model.norm.parameters():  # или model.head.norm, если есть
#     param.requires_grad = True

# 2. Классификационную голову (последний слой)
for param in model.head.parameters():
    param.requires_grad = True

In [ ]:
class Special_CNN_50000(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3, 12, kernel_size=3,padding=1, groups=1)
        self.bn1=nn.BatchNorm2d(12)
        self.relu1=nn.ReLU()

        self.conv2=nn.Conv2d(12, 36, kernel_size=3,padding=1,groups=12)
        self.bn2=nn.BatchNorm2d(36)
        self.relu2=nn.ReLU()

        self.pool1=nn.MaxPool2d(kernel_size=4, stride=4)

        self.conv3=nn.Conv2d(36, 108, kernel_size=3,padding=1,groups=36)
        self.bn3=nn.BatchNorm2d(108)
        self.relu3=nn.ReLU()


        self.conv4=nn.Conv2d(108, 108, kernel_size=3,padding=1,groups=108)
        self.bn4=nn.BatchNorm2d(108)
        self.relu4=nn.ReLU()

        self.conv5=nn.Conv2d(108, 216, kernel_size=3,padding=1,groups=108)
        self.bn5=nn.BatchNorm2d(216)
        self.relu5=nn.ReLU()

        self.conv6=nn.Conv2d(216, 216, kernel_size=3,padding=1,groups=216)
        self.bn6=nn.BatchNorm2d(216)
        self.relu6=nn.ReLU()

        self.conv7=nn.Conv2d(216, 216, kernel_size=3,padding=1,groups=108)
        self.bn7=nn.BatchNorm2d(216)
        self.relu7=nn.ReLU()

        self.conv8=nn.Conv2d(216, 216, kernel_size=3,padding=1,groups=216)
        self.bn8=nn.BatchNorm2d(216)
        self.relu8=nn.ReLU()

        self.pool2=nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv9=nn.Conv2d(216, 108, kernel_size=3,padding=1,bias=True,groups=108)
        self.bn9=nn.BatchNorm2d(108)
        self.relu9=nn.ReLU()

        self.conv10=nn.Conv2d(108, 108, kernel_size=3,padding=1,bias=True,groups=36)
        self.bn10=nn.BatchNorm2d(108)
        self.relu10=nn.ReLU()

        self.conv11=nn.Conv2d(108, 36, kernel_size=3,padding=1,bias=True,groups=36)
        self.bn11=nn.BatchNorm2d(36)
        self.relu11=nn.ReLU()

        self.pool3=nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv12=nn.Conv2d(36, 16, kernel_size=3,padding=1,groups=4)
        self.bn12=nn.BatchNorm2d(16)
        self.relu12=nn.ReLU()

        self.conv13=nn.Conv2d(16, 8, kernel_size=3,padding=1,groups=4)
        self.bn13=nn.BatchNorm2d(8)
        self.relu13=nn.ReLU()

        self.flatten=nn.Flatten()
        self.fc=nn.Linear(2048, 46)
        self.sm=nn.Softmax()

        self.alpha=nn.Parameter(torch.tensor(0.))
        self.beta=nn.Parameter(torch.tensor(0.))
        self.sigma=nn.Parameter(torch.tensor(0.))
    def forward(self, x):

        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu1(x)

        x=self.conv2(x)
        x=self.bn2(x)
        x=self.relu2(x)

        x=self.pool1(x)

        z=self.conv3(x)
        x=self.bn3(z)
        x=self.relu3(x)

        x=self.conv4(x)
        x=self.bn4(x+self.alpha*z)
        x=self.relu4(x)

        x=self.conv5(x)
        v=self.bn5(x)
        x=self.relu5(v)

        x=self.conv6(x)
        x=self.bn6(x)
        x=self.relu6(x)

        x=self.conv7(x)
        x=self.bn7(x)
        x=self.relu7(x)

        x=self.conv8(x)
        x=self.bn8(x)
        x=self.relu8(x+self.beta*v)

        x=self.pool2(x)

        x=self.conv9(x)
        o=self.bn9(x)
        x=self.relu9(o)

        x=self.conv10(x)
        x=self.bn10(x)
        x=self.relu10(x+self.sigma*o)

        x=self.conv11(x)
        x=self.bn11(x)
        x=self.relu11(x)

        x=self.pool3(x)

        x=self.conv12(x)
        x=self.bn12(x)
        x=self.relu12(x)

        x=self.conv13(x)
        x=self.bn13(x)
        x=self.relu13(x)



        x=self.flatten(x)
        # print(x.shape)
        x = self.fc(x)
        return x

In [ ]:
model=Special_CNN_50000()

In [ ]:
# import torch.nn as nn
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)


In [ ]:
train_model(
    model,
    X_train,
    Y_train,
    X_test,
    Y_test,
    criterion,
    optimizer,
    num_epochs=1000,
    device='cpu',
    batch_size=16,
    with_cv=False
  )

Validating: 100%|██████████| 81/81 [02:05<00:00,  1.55s/it, acc=1.17, loss=316]


Epoch 1: Train Loss: 3.9128, Train Acc: 1.21% |Train Loss: 3.9128, Train Acc: 1.17%


Epoch 2/1000 [Train]:  84%|████████▍ | 272/322 [31:46<06:03,  7.27s/it, acc=1.47, loss=1.25e+3]

In [ ]:
my_answ=np.empty(X_test.shape[0])
for i,pict in enumerate(X_test):
    batch=pict.unsqueeze(0)
    res=trained_model(batch.to('cuda')).argmax()
    my_answ[i]=int(int(res))

In [ ]:
my_answ

array([37.,  5.,  5., ..., 20.,  4., 20.])

In [ ]:
answ=pd.DataFrame(my_answ,columns=["label"])
answ.index = answ.index + 1
answ.to_csv('Kovalev_1_answ.csv',)